In [1]:
%load_ext autoreload
%autoreload 2

import dspy
import os
from dotenv import load_dotenv
load_dotenv()

dspy.configure(lm=dspy.OpenAI(model="gpt-4-0125-preview", max_tokens=4096))


CODEBASE_DIR_NAME = 'oa-small-dev'
GIT_PATH = '/Users/macbook/Documents/GitHub/'
CODEBASE_ROOT = os.path.join(GIT_PATH, CODEBASE_DIR_NAME)

In [2]:
from model import Repo
repo = Repo(root_path=CODEBASE_ROOT)

### Init Repo

Before Querying the repo we need to initialize the repo. During this initialization, a group of agents are going through the repo and building textual descriptions of the different componnents of the repo, these descriptions are then used by the agents to better navigate the repo.

When running for the first time on a repo, this initialization can take 5-10 minutes depending on the size of you repo... The descriptions are cached for the next runs

In [6]:
repo.init_repo()

### Query Repo

The query repo allows to directly ask a question to the repository. the crew is going to look for answers and output a complete description of what was found in the repo regarding your question.

In [14]:
repo_query_output = repo.query("What are the tools provided to the architect agent?")



> Entering new AgentExecutor chain...

Invoking: `Directory_expert_src` with `{'information_request': 'List the tools provided to the architect agent in the agents directory.'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_agents` with `{'information_request': 'List the tools provided to the architect agent in the agents directory.'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_architect` with `{'information_request': 'List all the tools provided to the architect agent in the agents directory.'}`




> Entering new AgentExecutor chain...

Invoking: `Python_file_expert___init___py` with `{'pyfile_request_question': 'List all the tools provided to the architect agent in the agents directory.'}`


content='The tools provided to the Architect agent in the `oa-small-dev/src/agents/architect/__init__.py` file are:\n\n1. **create_subtasks**:\n   - Description: This function is responsible for breaking up the overall project description in

In [15]:
print(repo_query_output['output'])

The tools provided to the Architect agent in the `oa-small-dev/src/agents/architect/__init__.py` file are:

1. **create_subtasks**:
   - Description: This function is responsible for breaking up the overall project description into multiple smaller tasks and then getting user feedback on them.
   - Relevant code snippet:
     ```python
     {
         "type": "function",
         "function": {
             "name": "create_subtasks",
             "description": "If the user asks to break it down into parts, then call create_subtasks. Based on the user's initial descriptions of the task, break the task down into detailed subtasks to accomplish the larger task. Each subtask should include a title and a detailed description of the subtask.",
             "parameters": {"type": "object", "properties": {}, "required": []},
         },
     }
     ```

2. **create_tasks**:
   - Description: This function is responsible for creating multiple tickets in parallel based on the subtasks generated 

### Query Repo architect

We add a architect agent to the repo. Query the architect to get a description of tasks and substacks to solve a ticket on the repo.

In [4]:
architect_output = repo.query_architect("Create a configuration to easily change the llm models used by the architect.")




> Entering new AgentExecutor chain...

Invoking: `oa-small-dev_Repo_expert` with `{'information_request': 'Explain the current setup for changing the language models used by the architect in the oa-small-dev repository.'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_src` with `{'information_request': 'Explain the setup for changing the language models used by the architect.'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_agents` with `{'information_request': 'Explain the setup for changing the language models used by the architect.'}`




> Entering new AgentExecutor chain...

Invoking: `Directory_expert_architect` with `{'information_request': 'Explain the setup for changing the language models used by the architect.'}`




> Entering new AgentExecutor chain...

Invoking: `Python_file_expert___init___py` with `{'pyfile_request_question': 'Explain the setup for changing the language models used by the architect.'}`


content="The s

In [9]:
print(architect_output['output'])

To create a configuration to easily change the language models used by the architect in the `oa-small-dev` repository, you can follow these tasks and subtasks:

### Tasks:
1. **Understand the Current Setup:**
   - Review the existing setup for changing language models in the `oa-small-dev` repository.
   - Explore the `__init__.py` file in the `architect` directory to understand the key functions involved.

2. **Identify Configuration Points:**
   - Determine the specific areas in the codebase where language models are defined or referenced.
   - Identify the variables or settings that control the selection of language models.

3. **Implement Configurable Model Selection:**
   - Modify the code to introduce a configuration mechanism for selecting language models.
   - Create a function or module that allows users to easily specify the desired language model.

4. **Update Documentation:**
   - Document the new configuration options for changing language models.
   - Provide clear instru

### Visualize conv

In [10]:
output = architect_output
output

{'input': 'Create a configuration to easily change the llm models used by the architect.',
 'output': "To create a configuration to easily change the language models used by the architect in the `oa-small-dev` repository, you can follow these tasks and subtasks:\n\n### Tasks:\n1. **Understand the Current Setup:**\n   - Review the existing setup for changing language models in the `oa-small-dev` repository.\n   - Explore the `__init__.py` file in the `architect` directory to understand the key functions involved.\n\n2. **Identify Configuration Points:**\n   - Determine the specific areas in the codebase where language models are defined or referenced.\n   - Identify the variables or settings that control the selection of language models.\n\n3. **Implement Configurable Model Selection:**\n   - Modify the code to introduce a configuration mechanism for selecting language models.\n   - Create a function or module that allows users to easily specify the desired language model.\n\n4. **Updat

In [12]:
def _print_steps(steps, level=0):
    tab = """    """
    for i, (tool, step) in enumerate(steps):
        # print(tab*level + "Step", i)
        if isinstance(step, dict):
            print(tab*level + "Tool : ", tool.tool)
            print(tab*level + "Tool input : ", step['input'])
            sub_steps = step['intermediate_steps']
            _print_steps(sub_steps, level+1)
        else:
            print(tab*level + "Tool : ", tool.tool)
            # print("ai message :", step.content)



In [13]:
_print_steps(output["intermediate_steps"])

Tool :  oa-small-dev_Repo_expert
Tool input :  Explain the current setup for changing the language models used by the architect in the oa-small-dev repository.
    Tool :  Directory_expert_src
    Tool input :  Explain the setup for changing the language models used by the architect.
        Tool :  Directory_expert_agents
        Tool input :  Explain the setup for changing the language models used by the architect.
            Tool :  Directory_expert_architect
            Tool input :  Explain the setup for changing the language models used by the architect.
                Tool :  Python_file_expert___init___py


In [ ]:
repo.children[0].children

[Directory(name=agents), Directory(name=helpers)]